## Pre-MCMC calculations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import astropy
import scipy.integrate as integrate
import pandas

In [2]:
# Importing constants 

Planck = astropy.cosmology.realizations.Planck18

T_CMB = Planck.Tcmb0  # Temperature of the CMB
H_0 = Planck.H0  # Current Hubble constant
Omega_m = Planck.Om0  # Matter density parameter
Omega_lambda = Planck.Ode0  # Dark energy density parameter
c = astropy.constants.c.to('km/s')  # Speed of light in km/s
sigma_T = astropy.constants.sigma_T.to('km2')  # Thomson scattering cross-section in km^2


# Setting parameters

z = 8  # Redshift of the quasar bubbles 
f_H = 0.21  # fraction of neutral hydrogen at z = 8

In [3]:
# Input Data

data = pandas.read_csv('Data/u_0.0pc_noise.csv')

Ang_deg = np.array(data['radius_y'])  # In arcmins
amp = np.array(data['amp_y']) * 1e-6  # Amplitude in K

In [4]:
# Defines the integrand that is to be integrated 
def integrand(z, Omega_m = Omega_m, Omega_lambda = Omega_lambda):
    return 1 / np.sqrt(Omega_m * (1 + z)**3 + Omega_lambda)

# Calculating other relevant distances

D_H = c / H_0  # Hubble distance

Integral = integrate.quad(integrand, 0, z)[0]  # Integrating from 0 to z
D_C = D_H * Integral  # Comoving distance
D_A = D_C / (1 + z)  # Angular diameter distance


# Function that converts the Angular radius into the actual radius

def AngToDist(Ang_deg, D_A = D_A):  # THE FUNCTION INPUTS ANGLES IN ARCMINS
     
    Ang_rad = Ang_deg * np.pi / (180 * 60)  # Convert arcmins to radians
    Distance = D_A * Ang_rad  # Convert angular distance to actual distance

    return Distance


In [5]:
# Calculating the actual radii of the bubbles and y

Radius = AngToDist(Ang_deg)  # in Mpc

y = amp / T_CMB.value  

In [6]:
# Calculating n_e and n_H

n_e = y * 3 * np.sqrt(1+z) / (sigma_T * 0.001 * 2 * Radius.to('km'))

n_H = n_e / (1 - f_H)  # Assuming all electrons are from ionized hydrogen

----

## MCMC Sampling

In [7]:
# Defining the model using the Stromgren sphere approximation

def model(theta, n_H): 
    N_dot, t_Q = theta
    return (((3*N_dot*t_Q)/(4*np.pi*n_H))**(1/3)) * (1+z)**(-1)

In [8]:
# Setting up initial, lower and upper bounds for the parameters

N_dot_ini = 10**58  # in s^-1
N_dot_min =  10**56  # in s^-1
N_dot_max = 10**60  # in s^-1

t_Q_ini = 10**7  # in years
t_Q_ini = t_Q_ini * 3.156e7  # in s
t_Q_min = 10**5  # in years
t_Q_min = t_Q_min * 3.156e7  # in s 
t_Q_max = 10**9  # in years
t_Q_max = t_Q_max * 3.156e7  # in s

In [9]:
# Calculating error in the diameter

Ang_err = np.array([0.1] * len(Ang_deg))  # Error in determining the angular diameter in arcmins
Radius_err = Ang_err * np.pi / (180 * 60) * D_A  # Convert angular error to actual diameter error

In [10]:
# Defining the log-likelihood function

def lnlike(theta, x, y, y_err):
    return -0.5 * np.sum(((y - model(theta, x)) / y_err) ** 2)

# Defining the log-prior function

def lnprior(theta):
    N_dot, t_Q = theta
    if N_dot < N_dot_min or t_Q < t_Q_min: # or N_dot > N_dot_max  or t_Q > t_Q_max:
        return -np.inf
    return 0.0

# Defining the log-posterior function

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)


In [ ]:
# Setting initial values for the parameters
initial = [N_dot_ini, t_Q_ini]  

# Setting up the MCMC sampler
ndim = len(initial)
nwalkers = 5 * ndim    # Preferentially 3-5 times the number of dimensions (minimum 2 times the number of dimensions)
n_burn = 200
n_steps = 10000

# Initializing the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(n_H, Radius, Radius_err))

# Adding little variation to the initial positions
p0 = []
for i in range(nwalkers):
    pos = [N_dot_ini, t_Q_ini]  # Start with the initial values
    pos[0] = pos[0] + 1e-4 * pos[0] * np.random.random(1)[0]  # Adding a small random variation to N_dot
    pos[1] = pos[1] + 1e-4 * pos[1] * np.random.random(1)[0]  # Adding a small random variation to t_Q
    p0.append(pos)   


In [12]:
# Running the MCMC sampler

# Burn-in phase
print("Running burn-in phase...")
sampler.reset()
state = sampler.run_mcmc(p0, n_burn, progress=True)
sampler.reset()

# Main sampling phase
print("Running main sampling phase...")
pos, prob, state = sampler.run_mcmc(state, n_steps, progress=True)

# Extracting the samples
samples = sampler.get_chain(flat=True)      # Flatten the chain to get all samples in a single array


Running burn-in phase...


100%|██████████| 20/20 [00:00<00:00, 373.91it/s]


Running main sampling phase...


100%|██████████| 1000/1000 [00:02<00:00, 436.18it/s]


In [13]:
theta_max = samples[np.argmax(sampler.flatlnprobability)]
print("N_dot:", theta_max[0], "; t_Q (in years):", theta_max[1]/3.156e7 )

N_dot: 6.707642251874908e+60 ; t_Q (in years): 424694950.82356524


In [14]:
# The Radii calculated from the model using the maximum likelihood parameters
model(theta_max, n_H).to('Mpc')

<Quantity [11.52989377, 11.32351927, 11.60293984, 11.86952971, 11.8852179 ,
           12.02594347, 12.88158861, 11.852531  ] Mpc>

In [15]:
# The radius_y values
Radius

<Quantity [10.64870116, 11.24029567, 11.53609292, 11.53609292, 11.83189018,
           12.42348469, 12.71928194, 13.0150792 ] Mpc>

In [16]:
sampler.acceptance_fraction

array([0.168, 0.169, 0.082, 0.108, 0.125, 0.103, 0.133, 0.18 , 0.144,
       0.164])

---

## Comparison with the input data set


Here I am performing the same MCMC sampling but with the input data, to compare the output with that of the previous scenario

In [17]:
# Importing the input data
ang_inp = np.array(data.radius_x)  # Input angular radius
                                            
amp_inp = np.array(data.amp_x) * -1 * 1e-6  # The input amplitude is negative, so we multiply it with -1 (in Kelvin)

In [18]:
# Calculating the actual distance and y

Radius_inp = AngToDist(ang_inp)  # in Mpc

y_inp = amp_inp / T_CMB.value

In [19]:
# Calculating n_e and n_H

n_e_inp = y * 3 * np.sqrt(1+z) / (sigma_T * 0.001 * 2 * Radius_inp.to('km'))

n_H_inp = n_e_inp / (1 - f_H) 

In [20]:
# Initializing the sampler for the input data

sampler_inp = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(n_H_inp, Radius_inp, Radius_err))


# Running the MCMC Sampler

# Burn-in phase
print("Running burn-in phase...")
sampler_inp.reset()
state = sampler_inp.run_mcmc(p0, n_burn, progress=True)
sampler_inp.reset()

# Main sampling phase
print("Running main sampling phase...")
pos, prob, state = sampler_inp.run_mcmc(state, n_steps, progress=True)

# Extracting the samples
samples = sampler_inp.get_chain(flat=True) 


Running burn-in phase...


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 320.92it/s]


Running main sampling phase...


100%|██████████| 1000/1000 [00:02<00:00, 391.35it/s]


In [21]:
# Comparing the N_dot and t_Q values from the input data with the output data
theta_max_inp = samples[np.argmax(sampler_inp.flatlnprobability)]
print("INPUT:  N_dot:", theta_max_inp[0], "; t_Q (in years):", theta_max_inp[1]/3.156e7 )
print("OUTPUT: N_dot:", theta_max[0], "; t_Q (in years):", theta_max[1]/3.156e7 )

INPUT:  N_dot: 4.866780555964437e+60 ; t_Q (in years): 585336231.1163474
OUTPUT: N_dot: 6.707642251874908e+60 ; t_Q (in years): 424694950.82356524


In [22]:
# Comparing the Radii calculated from the input and output data
print("INPUT: Radius:", model(theta_max_inp, n_H_inp).to('Mpc'))
print("OUTPUT: Radius:", model(theta_max, n_H).to('Mpc'))

INPUT: Radius: [11.52989538 11.32352085 11.60294146 11.86953136 11.88521955 12.02594515
 12.88159041 11.85253265] Mpc
OUTPUT: Radius: [11.52989377 11.32351927 11.60293984 11.86952971 11.8852179  12.02594347
 12.88158861 11.852531  ] Mpc


In [23]:
# Actual Radius values
print("Actual Radius:", Radius_inp)


Actual Radius: [10.64870116 11.24029567 11.53609292 11.53609292 11.83189018 12.42348469
 12.71928194 13.0150792 ] Mpc
